In [43]:
import yaml
import pandas as pd 
import matplotlib.pyplot as plt
controls = ['negative_control','positive_control']
grn_models = ['collectri','granie', 'figr', 'celloracle', 'scglue', 'scenicplus']

## GRN inference: multiomics

### scGLUE

In [1]:
!aws s3 sync s3://openproblems-data/resources/grn/results/scglue ./resources/results/scglue

download: s3://openproblems-data/resources/grn/results/scglue/state.yaml to resources/results/scglue/state.yaml
download: s3://openproblems-data/resources/grn/results/scglue/trace.txt to resources/results/scglue/trace.txt
download: s3://openproblems-data/resources/grn/results/scglue/output/grn.csv to resources/results/scglue/output/grn.csv


In [3]:
!viash run src/exp_analysis/config.vsh.yaml -- --tf_gene_net resources/results/scglue/output/grn.csv

Reading input files
Calculate basic stats
Outputting stats to : /viash_automount/mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/output/stats.json
Topological analysis
Plotting tf-gene in degree, dir:  /viash_automount/mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/output/tf_gene_indegree.png
Plotting tf-gene out degree, dir:  /viash_automount/mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/output/tf_gene_outdegree.png


## GRN evaluation for multiomics methods

In [1]:
!viash ns build --setup cb -q run_grn_evaluation

temporaryFolder: /tmp/viash_hub_repo10489538114835231235 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
Exporting run_grn_evaluation (workflows) =nextflow=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/nextflow/workflows/run_grn_evaluation
Not all configs built successfully
  39 configs were disabled
  1/1 configs built successfully


In [8]:
!bash scripts/run_grn_evaluation.sh "ridge"


In [9]:
!aws s3 sync s3://openproblems-data/resources/grn/results/grn_evaluation_ridge ./resources/results/grn_evaluation_ridge

download: s3://openproblems-data/resources/grn/results/grn_evaluation_ridge/scores.yaml to resources/results/grn_evaluation_ridge/scores.yaml
download: s3://openproblems-data/resources/grn/results/grn_evaluation_ridge/trace.txt to resources/results/grn_evaluation_ridge/trace.txt
download: s3://openproblems-data/resources/grn/results/grn_evaluation_ridge/metric_configs.yaml to resources/results/grn_evaluation_ridge/metric_configs.yaml


# Robustness analysis

In [1]:
# !viash ns build --setup cb --parallel
!viash ns build --setup cb -q run_robustness_analysis


temporaryFolder: /tmp/viash_hub_repo10268443552668753296 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
Exporting run_robustness_analysis (workflows) =nextflow=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/nextflow/workflows/run_robustness_analysis
Not all configs built successfully
  39 configs were disabled
  1/1 configs built successfully


## Shuffle net

In [3]:
!bash scripts/run_robust_analys.sh "net"

In [7]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_net ./resources/results/robust_analy_reg2_net

download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/state.yaml to resources/results/robust_analy_reg2_net/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/scores.yaml to resources/results/robust_analy_reg2_net/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/metric_configs.yaml to resources/results/robust_analy_reg2_net/metric_configs.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_net/trace.txt to resources/results/robust_analy_reg2_net/trace.txt


## Add noise to weights

In [41]:
!bash scripts/run_robust_analys.sh "weight"

weight
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [dreamy_davinci] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (1)
[5f/aa4675] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f3/689356] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustn

In [9]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight ./resources/results/robust_analy_reg2_weight

download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/scores.yaml to resources/results/robust_analy_reg2_weight/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/state.yaml to resources/results/robust_analy_reg2_weight/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/metric_configs.yaml to resources/results/robust_analy_reg2_weight/metric_configs.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_weight/trace.txt to resources/results/robust_analy_reg2_weight/trace.txt


## Negative vs positive controls

In [8]:
!bash scripts/run_pc_vs_nc.sh "pc_vs_nc"

In [12]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_pc_vs_nc ./resources/results/robust_analy_reg2_pc_vs_nc

## regulatory sign

In [50]:
!bash scripts/run_robust_analys.sh "sign"

sign
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [soggy_sinoussi] DSL2 - revision: d55e1b9116
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (2)
[2f/3f5479] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f9/652d7d] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustne

In [10]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign ./resources/results/robust_analy_reg2_sign

download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/trace.txt to resources/results/robust_analy_reg2_sign/trace.txt
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/state.yaml to resources/results/robust_analy_reg2_sign/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/scores.yaml to resources/results/robust_analy_reg2_sign/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_reg2_sign/metric_configs.yaml to resources/results/robust_analy_reg2_sign/metric_configs.yaml


## Corr vs causal

In [ ]:
!bash scripts/run_robust_analys_causal.sh 

In [3]:
!aws s3 sync s3://openproblems-data/resources/grn/results/robust_analy_causal ./resources/results/robust_analy_causal

download: s3://openproblems-data/resources/grn/results/robust_analy_causal/state.yaml to resources/results/robust_analy_causal/state.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_causal/scores.yaml to resources/results/robust_analy_causal/scores.yaml
download: s3://openproblems-data/resources/grn/results/robust_analy_causal/trace.txt to resources/results/robust_analy_causal/trace.txt
download: s3://openproblems-data/resources/grn/results/robust_analy_causal/metric_configs.yaml to resources/results/robust_analy_causal/metric_configs.yaml


reg1-corr-5
reg1-corr-3
reg1-corr-2
reg1-corr-10
reg1-corr-1
reg1-corr-9
reg1-corr-8
reg1-corr-6
reg1-corr-7
reg1-corr-4


,ex(False)_tf(-1),ex(True)_tf(-1),Mean
corr_5,0.160433,0.168758,0.164596
corr_3,0.191852,0.202196,0.197024
corr_2,0.202665,0.213507,0.208086
corr_10,0.166490,0.175233,0.170861
corr_1,0.222673,0.234688,0.228680
corr_9,0.182540,0.192268,0.187404
corr_8,0.186710,0.196692,0.191701
corr_6,0.217203,0.229086,0.223144
corr_7,0.238084,0.250973,0.244528
corr_4,0.178851,0.187996,0.183423


In [13]:
!viash run src/robustness_analysis/causal/config.vsh.yaml -- --causal true 
!viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/prediction.csv

Read data

Noramlize data
Create corr net
Processing groups: 100%|██████████| 4/4 [00:41<00:00, 10.47s/it]
Output GRN


In [56]:
# 
corr_scores = []
for i in range(1, n_iter):    
    corr_scores.append(ad.read_h5ad(f"output/causal/scores/net_corr_{i}.h5ad").uns['metric_values'][0])
pc_score = ad.read_h5ad(f"output/causal/scores/net_pc.h5ad").uns['metric_values'][0]

In [59]:
(corr_scores<pc_score).sum()

95

## Corr grn from multiomics

In [69]:
multiomics_rna = ad.read_h5ad('resources/grn-benchmark/multiomics_rna.h5ad')

In [70]:
import scanpy as sc 
sc.pp.normalize_total(multiomics_rna)
sc.pp.log1p(multiomics_rna)
sc.pp.scale(multiomics_rna)
X = multiomics_rna.X

In [75]:
def corr_grn(X: np.ndarray, groups: np.ndarray):
    grns = []
    for group in tqdm(np.unique(groups), desc="Processing groups"):
        X_sub = X[groups == group, :]
        X_sub = StandardScaler().fit_transform(X_sub)
        grn = np.dot(X_sub.T, X_sub) / X_sub.shape[0]
        grns.append(grn)
    return np.mean(grns, axis=0)
groups = multiomics_rna.obs.cell_type
corr_net = corr_grn(X, groups)

Processing groups: 100%|██████████| 4/4 [00:43<00:00, 10.93s/it]


In [79]:
corr_net = pd.DataFrame(corr_net, index=multiomics_rna.var_names, columns=multiomics_rna.var_names)

In [99]:
tfs = corr_net.abs().sum(axis=0).argsort()[::-1][:1000].index.to_numpy()
corr_net_sub = corr_net[tfs]

In [100]:
corr_net_sub = corr_net_sub.reset_index().melt(id_vars='location', var_name='source', value_name='weight')
corr_net_sub.rename(columns={'location': 'target'}, inplace=True)
corr_net_sub.to_csv('output/causal/grns/corr_net_sub.csv')

In [101]:
cmd = f"viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/causal/grns/corr_net_sub.csv --score output/causal/scores/corr_net_sub.h5ad"
subprocess.run(cmd, shell=True)

Reading input files
(549, 15215)
Compute metrics for layer: pearson
ex(False)_tf(-1)

Processing groups: 100%|██████████| 5/5 [00:02<00:00,  2.35it/s]
/viash_automount/tmp/viash-run-regression_1-Itbpvt.py:48: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  output = ad.AnnData(
   ex(False)_tf(-1)      Mean
0          0.118948  0.118948
(2,) (2,)


CompletedProcess(args='viash run src/metrics/regression_1/config.vsh.yaml -- --prediction output/causal/grns/corr_net_sub.csv --score output/causal/scores/corr_net_sub.h5ad', returncode=0)

# Sinlge omics

In [ ]:
!bash scripts/run_benchmark_single_omics.sh

In [2]:
!aws s3 sync s3://openproblems-data/resources_test/grn/results/single_omics_all_test ./resources_test/results/single_omics_all_test

download: s3://openproblems-data/resources_test/grn/results/single_omics_all_test/state.yaml to resources_test/results/single_omics_all_test/state.yaml
download: s3://openproblems-data/resources_test/grn/results/single_omics_all_test/scores.yaml to resources_test/results/single_omics_all_test/scores.yaml
download: s3://openproblems-data/resources_test/grn/results/single_omics_all_test/trace.txt to resources_test/results/single_omics_all_test/trace.txt
download: s3://openproblems-data/resources_test/grn/results/single_omics_all_test/ridge.grnboost2.grnboost2.prediction.csv to resources_test/results/single_omics_all_test/ridge.grnboost2.grnboost2.prediction.csv
download: s3://openproblems-data/resources_test/grn/results/single_omics_all_test/ridge.ppcor.ppcor.prediction.csv to resources_test/results/single_omics_all_test/ridge.ppcor.ppcor.prediction.csv
download: s3://openproblems-data/resources_test/grn/results/single_omics_all_test/ridge.scsgl.scsgl.prediction.csv to resources_test/res

In [ ]:
def extract_data(data, reg='reg1'):
    i = 0
    for entry in data:
        rg, method_id = entry['method_id'].split('-')
        if rg != reg:
            continue
        dataset_id = entry['dataset_id']
        metric_ids = entry['metric_ids']
        metric_values = entry['metric_values']
        
        df = pd.DataFrame([metric_values], index=[method_id], columns=metric_ids)
        if i==0:
            df_reg = df
        else:
            df_reg = pd.concat([df_reg, df], axis=0)
        i+=1
    return df_reg
df_reg1 = extract_data(data, reg='reg1')
df_reg2 = extract_data(data, reg='reg2')